<a href="https://colab.research.google.com/github/Baludatascience/Sentiment-Analysis-using-LSTM/blob/master/LSTM_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
##Import necessary packages/libraries 

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

In [6]:
data = pd.read_csv(r"C:\Users\HP\Downloads\Sentiment-Analysis-using-LSTM-master\Sentiment-Analysis-using-LSTM-master\NewSentiment5.csv",encoding='latin-1')

 ## reading the data from folder

#Considering only the necessary columns#
data = data[['SentimentText', 'Sentiment']]
data.shape     

(152939, 2)

In [7]:
# Performing data preprocessing##

data  = data[data.Sentiment != 'Netural']   ## considering only "Positive & Negative" sentiments
data ['SentimentText']  = data['SentimentText'].apply(lambda x: x.lower())
data ['SentimentText'] = data['SentimentText'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x)))

print( data[data['Sentiment'] == 'Positive'].size)
print( data[data['Sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
  row[0] =row[0].replace('rt', ' ') ##

max_fatures = 20000
tokenizer = Tokenizer(num_words = max_fatures, split= ' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)



<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\HP\AppData\Local\Temp\ipykernel_1332\916500599.py:5: SyntaxWarning: invalid escape sequence '\s'
  data ['SentimentText'] = data['SentimentText'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x)))


175944
129934


C:\Users\HP\AppData\Local\Temp\ipykernel_1332\916500599.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] =row[0].replace('rt', ' ') ##
C:\Users\HP\AppData\Local\Temp\ipykernel_1332\916500599.py:11: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] =row[0].replace('rt', ' ') ##


In [8]:
#Creating layers using LSTM
embed_dim = 128    
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.33, random_state= 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(102469, 137) (102469, 2)
(50470, 137) (50470, 2)


In [10]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2)

Epoch 1/3
3203/3203 - 488s - 152ms/step - accuracy: 0.7543 - loss: 0.5004
Epoch 2/3
3203/3203 - 457s - 143ms/step - accuracy: 0.8034 - loss: 0.4235
Epoch 3/3
3203/3203 - 432s - 135ms/step - accuracy: 0.8275 - loss: 0.3811


In [7]:
!output_directory new/

/bin/bash: output_directory: command not found


In [11]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

1531/1531 - 68s - 45ms/step - accuracy: 0.7843 - loss: 0.4578
score: 0.46
acc: 0.78


In [12]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 337ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 40ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 45ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 47ms/step
1/1 - 0s - 40ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 46ms/step
1/1 - 0s - 50ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 49ms/step
1/1 - 0s - 48ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 40ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 40ms/step
1/1 - 0s - 40ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 39ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 44ms/step
1/1 - 0s - 41ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 44ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 43ms/step
1/1 - 0s - 4

In [31]:
##
twt  = ['This project is truly outstanding! The level of effort, attention to detail, and creativity you’ve demonstrated are remarkable, and its clear that you put a lot of thought and dedication into making this as impactful and polished as it is fantastic work all around!']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=30, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[ 1291     4  5624     7  8408    29     7    84  1565     8     2   315
      6   257    16   206     7 10218   275   428    46    71     7    71
      5    14   915    83    40   297]]
1/1 - 0s - 31ms/step
1/1 - 0s - 33ms/step
positive
